In [22]:
import findspark
findspark.init()


In [23]:
from pyspark import SparkContext

In [33]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("CrimeDataAnalysis") \
    .getOrCreate()

# Now you can use the 'spark' variable to read CSV files or perform other Spark operations


In [27]:
!wget https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/

--2024-05-11 17:58:30--  https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-05-11 17:58:30 ERROR 404: Not Found.



In [36]:
from pyspark.sql.functions import to_timestamp,col,lit
read_csv = spark.read.csv('sample_data/crime.csv', header=True)
read_csv.show(5)


+--------+--------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#| DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+--------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG497095|11/08/2023 08:50:...| 025XX N KEDZIE BLVD| 0810|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|
|JG496991|11/08/2023 03:14:...| 0000X W CHICAGO AVE| 0560|             A